In [1]:
import numpy as np
import pandas as pd

In [2]:
batch1 = pd.read_parquet('../batches_train/batch_1_repartitions/part.0.parquet')
sensor_geom = pd.read_csv('../sensor_geometry.csv')

def id_to_xyz(sen):
    return sensor_geom.loc[sen][['x','y','z']].values

In [141]:
meta = pd.read_parquet('../batches_train/train_meta.parquet/')

In [155]:
meta.head()

,batch_id,event_id,first_pulse_index,last_pulse_index,azimuth,zenith
0,1,24,0,60,5.029555,2.087498
1,1,41,61,111,0.417742,1.549686
2,1,59,112,147,1.160466,2.401942
3,1,67,148,289,5.845952,0.759054
4,1,72,290,351,0.653719,0.939117


In [100]:
# Physics papers tell us how to find the 'best-fit' line
# r = r_0 + v*t, with r_0 and v calculated as follows
def find_direction(event):
    placetimes = event.drop_duplicates('sensor_id')[['sensor_id', 'time']].reset_index()
    for i in range(0,len(placetimes)):
        placetimes.loc[i,['x','y','z']] = id_to_xyz(placetimes.loc[i,'sensor_id'])
        placetimes.loc[i,['xt','yt','zt']] = (placetimes.loc[i, ['x','y','z']]*placetimes.loc[i,'time']).values
        placetimes.loc[i,'time_squared'] = np.square(placetimes.loc[i,'time'])
        
    means = placetimes.mean(axis=0)
    
    vx = (means.xt - means.x*means.time)/(means.time_squared - np.square(means.time))
    vy = (means.yt - means.y*means.time)/(means.time_squared - np.square(means.time))
    vz = (means.zt - means.z*means.time)/(means.time_squared - np.square(means.time))

    v = -np.array([vx,vy,vz])
    
    return v/np.linalg.norm(v)

In [101]:
find_direction(batch1.loc[24])

array([0.59365092, 0.42984057, 0.68030557])

In [129]:
def vector_to_angle(v):
    v = np.clip(v, -1, 1)
    ze = np.arccos(v[2])
    az = np.arctan2(v[1],v[0])
    return (az,ze)

In [163]:
def find_angular_origin(event):
    cartesian_direction = find_direction(event)
    prelim = vector_to_angle(cartesian_direction)
    az = prelim[0]
    ze = prelim[1]
    
    if az < 0.001:
        az = 2*np.pi - az
    
    return (az, ze)

In [159]:
find_angular_origin(batch1.loc[24])

(0.6266936314955205, 0.8226168529759329)

### Testing

In [160]:
event_ids = list(set(batch1.index))

In [161]:
N = 100
test_indices = [event_ids[i] for i in range(0,N)]

meta_subset = meta[meta.event_id.isin(test_indices)]
batch1_subset = batch1.loc[test_indices]

In [164]:
preds = []
trues = []

count = 0
for index in test_indices:
    count = count + 1
    print("Testing..." + str(count), "complete")
    preds.append(find_angular_origin(batch1_subset.loc[index]))
    
    ev = meta_subset[meta_subset.event_id==index]
    trues.append((ev.azimuth.values[0], ev.zenith.values[0]))

Testing...1 complete
Testing...2 complete
Testing...3 complete
Testing...4 complete
Testing...5 complete
Testing...6 complete
Testing...7 complete
Testing...8 complete
Testing...9 complete
Testing...10 complete
Testing...11 complete
Testing...12 complete
Testing...13 complete
Testing...14 complete
Testing...15 complete
Testing...16 complete
Testing...17 complete
Testing...18 complete
Testing...19 complete
Testing...20 complete
Testing...21 complete
Testing...22 complete
Testing...23 complete
Testing...24 complete
Testing...25 complete
Testing...26 complete
Testing...27 complete
Testing...28 complete
Testing...29 complete
Testing...30 complete
Testing...31 complete
Testing...32 complete
Testing...33 complete
Testing...34 complete
Testing...35 complete
Testing...36 complete
Testing...37 complete
Testing...38 complete
Testing...39 complete
Testing...40 complete
Testing...41 complete
Testing...42 complete
Testing...43 complete
Testing...44 complete
Testing...45 complete
Testing...46 comple

In [165]:
def mse(A, B):
    D = np.zeros(A.shape)
    for i in range(0, D.shape[0]):
        D[i,0] = min(np.abs(A[i,0] - B[i,0]),
                     np.abs(A[i,0] - B[i,0] - 2*np.pi),
                     np.abs(A[i,0] - B[i,0] + 2*np.pi))
        D[i,1] = min(np.abs(A[i,1] - B[i,1]),
                     np.abs(A[i,1] - B[i,1] - np.pi),
                     np.abs(A[i,1] - B[i,1] - np.pi))
    return (np.square(D)).mean(axis=0)

In [166]:
mse(np.array(preds), np.array(trues))

array([3.23222768, 0.97845222])